In [1]:
import os
import sys

src_path = os.path.abspath('../..')
print(src_path)
sys.path.append(src_path)

/home/zw12/missing_data


In [2]:
from src.utils import create_directory, dump_pickle, raw_data_path, processed_data_path, set_seed

In [3]:
import csv
import json
from datetime import datetime, timedelta
from collections import Counter, OrderedDict
import random
import pandas as pd
import numpy as np
from itertools import chain

In [4]:
set_seed(seed=42)

In [5]:
input_path = os.path.join(raw_data_path, "physionet.org/files/eicu-crd/2.0")
output_path = os.path.join(processed_data_path, "eicu")
create_directory(output_path)

## lab

In [6]:
lab_raw = pd.read_csv(os.path.join(input_path, 'lab.csv'))
print(lab_raw.shape)
lab_raw.head()

(39132531, 10)


,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,52307161,141168,2026,3,fibrinogen,177.0,177,mg/dL,mg/dL,2219
1,50363251,141168,1133,3,PT - INR,2.5,2.5,ratio,NaN,1208
2,49149139,141168,2026,1,magnesium,2.0,2.0,mg/dL,mg/dL,2090
3,50363250,141168,1133,3,PT,26.6,26.6,sec,sec,1208
4,66695374,141168,2141,7,pH,7.2,7.20,NaN,Units,2155


In [7]:
lab_raw.labname.nunique()

158

In [10]:
lab_raw.groupby("labname").labmeasurenamesystem.nunique().unique()

array([1, 0])

In [13]:
lab_raw.groupby("labname").labmeasurenameinterface.nunique()

labname
-bands                       2
-basos                       7
-eos                         6
-lymphs                      6
-monos                       6
                            ..
uric acid                    3
urinary creatinine           5
urinary osmolality          21
urinary sodium              11
urinary specific gravity     2
Name: labmeasurenameinterface, Length: 158, dtype: int64

In [22]:
lab_raw[lab_raw.labname == "urinary osmolality"].labmeasurenameinterface.unique()

array(['mOsm/kg', 'mOsm/Kg', 'mosm/kg', 'mOSm/kg', 'mos/Kg', 'mosm/K',
       'mOsm/L', 'mOs/kg', nan, 'MOSM/KG', 'mOSM/KG', 'uosm/kg', 'mOsm',
       'MMOL/KG', 'mOsm/KG', 'mOsm/K', 'mOSM/kg', 'MOS/KG', 'mOsmol/kg',
       'mOs/kH2O', 'Mosm/kg', 'mOSM/kG'], dtype=object)

In [23]:
lab_raw[lab_raw.labname == "urinary osmolality"].labmeasurenamesystem.unique()

array(['mOsm/L'], dtype=object)

In [24]:
lab_raw[lab_raw.labname == "urinary osmolality"][lab_raw.labmeasurenameinterface == "mOsm/kg"]

/tmp/ipykernel_1006023/1889174300.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lab_raw[lab_raw.labname == "urinary osmolality"][lab_raw.labmeasurenameinterface == "mOsm/kg"]


,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
8734,57276354,141585,42502,4,urinary osmolality,317.0,317,mOsm/L,mOsm/kg,42577
18392,58145457,141924,12175,4,urinary osmolality,197.0,197,mOsm/L,mOsm/kg,12226
19948,58495129,141978,-1725,4,urinary osmolality,350.0,350,mOsm/L,mOsm/kg,-1666
20218,58152940,141979,18391,4,urinary osmolality,335.0,335,mOsm/L,mOsm/kg,18466
30559,57225630,142616,634,4,urinary osmolality,703.0,703,mOsm/L,mOsm/kg,728
...,...,...,...,...,...,...,...,...,...,...
39107415,827146683,3352637,5769,4,urinary osmolality,145.0,145,mOsm/L,mOsm/kg,5803
39116009,827200435,3352940,679,4,urinary osmolality,404.0,404,mOsm/L,mOsm/kg,744
39116302,827443483,3352940,28788,4,urinary osmolality,431.0,431,mOsm/L,mOsm/kg,28833
39121309,827170612,3353044,6158,4,urinary osmolality,386.0,386,mOsm/L,mOsm/kg,6200


In [25]:
lab_raw[lab_raw.labname == "urinary osmolality"][lab_raw.labmeasurenameinterface == "mOs/kH2O"]

/tmp/ipykernel_1006023/560137410.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lab_raw[lab_raw.labname == "urinary osmolality"][lab_raw.labmeasurenameinterface == "mOs/kH2O"]


,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
31663693,727145917,2904244,212,4,urinary osmolality,275.0,275,mOsm/L,mOs/kH2O,212
31668507,726808599,2904867,1604,4,urinary osmolality,99.0,99,mOsm/L,mOs/kH2O,1604
31668508,727271415,2904867,-10,4,urinary osmolality,110.0,110,mOsm/L,mOs/kH2O,-10
31676718,727508788,2905907,-294,4,urinary osmolality,451.0,451,mOsm/L,mOs/kH2O,-294
31687600,727516729,2907403,12600,4,urinary osmolality,605.0,605,mOsm/L,mOs/kH2O,12600
...,...,...,...,...,...,...,...,...,...,...
32221248,727140543,2984498,1346,4,urinary osmolality,615.0,615,mOsm/L,mOs/kH2O,1346
32222249,727500089,2984653,16503,4,urinary osmolality,577.0,577,mOsm/L,mOs/kH2O,16503
32222257,727533905,2984653,12223,4,urinary osmolality,518.0,518,mOsm/L,mOs/kH2O,12223
32237491,727557120,2986549,1360,4,urinary osmolality,364.0,364,mOsm/L,mOs/kH2O,1360


In [26]:
lab_raw.labresult.isna().sum()

229518

In [28]:
lab_raw = lab_raw.dropna(subset=["labresult"])
lab_raw["normalized_labresult"] = lab_raw.groupby('labname')['labresult'].transform(lambda x: (x - x.mean()) / x.std())
lab_raw

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset,normalized_labresult
0,52307161,141168,2026,3,fibrinogen,177.00,177,mg/dL,mg/dL,2219,-0.836610
1,50363251,141168,1133,3,PT - INR,2.50,2.5,ratio,NaN,1208,0.825760
2,49149139,141168,2026,1,magnesium,2.00,2.0,mg/dL,mg/dL,2090,0.032493
3,50363250,141168,1133,3,PT,26.60,26.6,sec,sec,1208,0.829304
4,66695374,141168,2141,7,pH,7.20,7.20,NaN,Units,2155,-0.016024
...,...,...,...,...,...,...,...,...,...,...,...
39132526,824772675,3353263,-7,3,WBC x 1000,6.40,6.4,K/mcL,K/uL,6,-0.606647
39132527,826470517,3353263,1733,3,RBC,4.67,4.67,M/mcL,M/uL,1774,1.509200
39132528,824772678,3353263,-7,3,-monos,10.00,10,%,%,6,0.489614
39132529,826470516,3353263,1733,3,WBC x 1000,6.60,6.6,K/mcL,K/uL,1774,-0.583142


In [29]:
output_path

'/srv/local/data/zw12/missing_data/processed_data/eicu'

In [30]:
lab_raw.to_csv(os.path.join(output_path, 'lab_tmp.csv'), index=False)

## apacheApsVar

In [79]:
apacheapsvar = pd.read_csv(os.path.join(input_path, 'apacheApsVar.csv'))
print(apacheapsvar.shape)
apacheapsvar.head()

(171177, 26)


,apacheapsvarid,patientunitstayid,intubated,vent,dialysis,eyes,motor,verbal,meds,urine,...,ph,hematocrit,creatinine,albumin,pao2,pco2,bun,glucose,bilirubin,fio2
0,2,141168,0,0,0,4,6,5,0,-1.0,...,-1.00,40.1,2.30,3.1,-1.0,-1.0,27.0,95.0,4.1,-1.0
1,2399659,141178,0,0,0,-1,-1,-1,-1,-1.0,...,-1.00,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,167885,141194,0,0,0,3,6,4,0,-1.0,...,-1.00,27.4,2.51,2.3,-1.0,-1.0,31.0,168.0,0.4,-1.0
3,70691,141197,0,0,0,4,6,5,0,-1.0,...,-1.00,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,2279689,141203,0,1,0,1,3,1,0,-1.0,...,7.45,36.9,0.56,-1.0,51.0,37.0,9.0,145.0,-1.0,100.0


In [80]:
apacheapsvar.isna().sum()

apacheapsvarid       0
patientunitstayid    0
intubated            0
vent                 0
dialysis             0
eyes                 0
motor                0
verbal               0
meds                 0
urine                0
wbc                  0
temperature          0
respiratoryrate      0
sodium               0
heartrate            0
meanbp               0
ph                   0
hematocrit           0
creatinine           0
albumin              0
pao2                 0
pco2                 0
bun                  0
glucose              0
bilirubin            0
fio2                 0
dtype: int64

In [81]:
apacheapsvar = apacheapsvar.replace(-1, float("nan"))
apacheapsvar

,apacheapsvarid,patientunitstayid,intubated,vent,dialysis,eyes,motor,verbal,meds,urine,...,ph,hematocrit,creatinine,albumin,pao2,pco2,bun,glucose,bilirubin,fio2
0,2,141168,0,0,0,4.0,6.0,5.0,0.0,NaN,...,NaN,40.1,2.30,3.1,NaN,NaN,27.0,95.0,4.1,NaN
1,2399659,141178,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,167885,141194,0,0,0,3.0,6.0,4.0,0.0,NaN,...,NaN,27.4,2.51,2.3,NaN,NaN,31.0,168.0,0.4,NaN
3,70691,141197,0,0,0,4.0,6.0,5.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2279689,141203,0,1,0,1.0,3.0,1.0,0.0,NaN,...,7.450,36.9,0.56,NaN,51.0,37.0,9.0,145.0,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,1325400,3353235,0,0,0,4.0,6.0,5.0,0.0,1751.1552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171173,1528627,3353237,0,0,0,4.0,6.0,4.0,0.0,5724.7776,...,NaN,39.0,1.07,3.5,NaN,NaN,32.0,139.0,0.7,NaN
171174,1546299,3353251,1,1,0,1.0,1.0,1.0,0.0,358.9920,...,7.224,NaN,2.43,NaN,80.0,44.0,30.0,346.0,NaN,100.0
171175,2085296,3353254,0,0,0,4.0,6.0,5.0,0.0,2171.6640,...,NaN,32.0,NaN,NaN,NaN,NaN,NaN,137.0,NaN,NaN


In [82]:
apacheapsvar = apacheapsvar
apacheapsvar['nan_rate'] = apacheapsvar.isna().sum(axis=1) / df.shape[1]

In [83]:
apacheapsvar[apacheapsvar.nan_rate > 0.50]

,apacheapsvarid,patientunitstayid,intubated,vent,dialysis,eyes,motor,verbal,meds,urine,...,hematocrit,creatinine,albumin,pao2,pco2,bun,glucose,bilirubin,fio2,nan_rate
1,2399659,141178,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667
3,70691,141197,0,0,0,4.0,6.0,5.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.555556
11,35348,141264,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.629630
47,1825120,141565,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.777778
65,4437,141719,0,0,0,4.0,6.0,5.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.518519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171036,1329795,3352439,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.777778
171045,1334221,3352481,0,0,0,4.0,6.0,5.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.629630
171049,2147142,3352494,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.777778
171154,1422593,3353123,0,1,0,1.0,1.0,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.555556


In [84]:
apacheapsvar = apacheapsvar[apacheapsvar.nan_rate <= 0.50]
apacheapsvar

,apacheapsvarid,patientunitstayid,intubated,vent,dialysis,eyes,motor,verbal,meds,urine,...,hematocrit,creatinine,albumin,pao2,pco2,bun,glucose,bilirubin,fio2,nan_rate
0,2,141168,0,0,0,4.0,6.0,5.0,0.0,NaN,...,40.1,2.30,3.1,NaN,NaN,27.0,95.0,4.1,NaN,0.185185
2,167885,141194,0,0,0,3.0,6.0,4.0,0.0,NaN,...,27.4,2.51,2.3,NaN,NaN,31.0,168.0,0.4,NaN,0.185185
4,2279689,141203,0,1,0,1.0,3.0,1.0,0.0,NaN,...,36.9,0.56,NaN,51.0,37.0,9.0,145.0,NaN,100.0,0.111111
5,123706,141208,0,0,0,3.0,6.0,5.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.481481
6,172304,141227,0,1,0,3.0,6.0,4.0,0.0,NaN,...,26.2,1.90,NaN,65.0,23.0,32.0,145.0,NaN,21.0,0.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,1325400,3353235,0,0,0,4.0,6.0,5.0,0.0,1751.1552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444444
171173,1528627,3353237,0,0,0,4.0,6.0,4.0,0.0,5724.7776,...,39.0,1.07,3.5,NaN,NaN,32.0,139.0,0.7,NaN,0.148148
171174,1546299,3353251,1,1,0,1.0,1.0,1.0,0.0,358.9920,...,NaN,2.43,NaN,80.0,44.0,30.0,346.0,NaN,100.0,0.148148
171175,2085296,3353254,0,0,0,4.0,6.0,5.0,0.0,2171.6640,...,32.0,NaN,NaN,NaN,NaN,NaN,137.0,NaN,NaN,0.370370


In [85]:
apacheapsvar.intubated.unique()

array([0, 1])

In [86]:
apacheapsvar.vent.unique()

array([0, 1])

In [87]:
apacheapsvar.dialysis.unique()

array([0, 1])

In [88]:
apacheapsvar.eyes.unique()

array([ 4.,  3.,  1.,  2., nan])

In [89]:
apacheapsvar.motor.unique()

array([ 6.,  3.,  1.,  4.,  5., nan,  2.])

In [90]:
apacheapsvar.verbal.unique()

array([ 5.,  4.,  1.,  3.,  2., nan])

In [91]:
apacheapsvar.meds.unique()

array([ 0.,  1., nan])

In [92]:
category = [
    "intubated",
    "vent",
    "dialysis",
    "eyes",
    "motor",
    "verbal",
    "meds",
]

continuous = [
    "urine",
    "wbc",
    "temperature",
    "respiratoryrate",
    "sodium",
    "heartrate",
    "meanbp",
    "ph",
    "hematocrit",
    "creatinine",
    "albumin",
    "pao2",
    "pco2",
    "bun",
    "glucose",
    "bilirubin",
    "fio2",
]

In [93]:
def process_df(df, category_col, continuous_col):
    # Fill NaNs in category columns with the mode (most frequent value)
    for col in category_col:
        df[col].fillna(df[col].mode()[0], inplace=True)
    
    # Normalize continuous columns
    for col in continuous_col:
        df[col] = (df[col] - df[col].mean()) / df[col].std()
        df[col].fillna(0.0, inplace=True)
    
    return df

In [94]:
apacheapsvar = process_df(apacheapsvar, category, continuous)
apacheapsvar

/tmp/ipykernel_1006023/3569029725.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipykernel_1006023/3569029725.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (df[col] - df[col].mean()) / df[col].std()
/tmp/ipykernel_1006023/3569029725.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(0.0, inp

,apacheapsvarid,patientunitstayid,intubated,vent,dialysis,eyes,motor,verbal,meds,urine,...,hematocrit,creatinine,albumin,pao2,pco2,bun,glucose,bilirubin,fio2,nan_rate
0,2,141168,0,0,0,4.0,6.0,5.0,0.0,0.000000,...,1.087930,0.433971,0.344477,0.000000,0.000000,0.009185,-0.673780,1.172360,0.000000,0.185185
2,167885,141194,0,0,0,3.0,6.0,4.0,0.0,0.000000,...,-0.749805,0.555855,-0.806135,0.000000,0.000000,0.190668,0.046219,-0.336312,0.000000,0.185185
4,2279689,141203,0,1,0,1.0,3.0,1.0,0.0,0.000000,...,0.624879,-0.575929,0.000000,-0.931531,-0.439056,-0.807489,-0.180630,0.000000,1.537022,0.111111
5,123706,141208,0,0,0,3.0,6.0,5.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.481481
6,172304,141227,0,1,0,3.0,6.0,4.0,0.0,0.000000,...,-0.923449,0.201810,0.000000,-0.767675,-1.481805,0.236039,-0.180630,0.000000,-1.455272,0.111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,1325400,3353235,0,0,0,4.0,6.0,5.0,0.0,-0.034765,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.444444
171173,1528627,3353237,0,0,0,4.0,6.0,4.0,0.0,2.104829,...,0.928756,-0.279924,0.919783,0.000000,0.000000,0.236039,-0.239808,-0.213987,0.000000,0.148148
171174,1546299,3353251,1,1,0,1.0,1.0,1.0,0.0,-0.784374,...,0.000000,0.509423,0.000000,-0.592114,0.082318,0.145298,1.801835,0.000000,1.537022,0.148148
171175,2085296,3353254,0,0,0,4.0,6.0,5.0,0.0,0.191658,...,-0.084168,0.000000,0.000000,0.000000,0.000000,0.000000,-0.259534,0.000000,0.000000,0.370370


In [96]:
apacheapsvar = apacheapsvar.drop(columns=["nan_rate"])
apacheapsvar

,apacheapsvarid,patientunitstayid,intubated,vent,dialysis,eyes,motor,verbal,meds,urine,...,ph,hematocrit,creatinine,albumin,pao2,pco2,bun,glucose,bilirubin,fio2
0,2,141168,0,0,0,4.0,6.0,5.0,0.0,0.000000,...,0.000000,1.087930,0.433971,0.344477,0.000000,0.000000,0.009185,-0.673780,1.172360,0.000000
2,167885,141194,0,0,0,3.0,6.0,4.0,0.0,0.000000,...,0.000000,-0.749805,0.555855,-0.806135,0.000000,0.000000,0.190668,0.046219,-0.336312,0.000000
4,2279689,141203,0,1,0,1.0,3.0,1.0,0.0,0.000000,...,0.941532,0.624879,-0.575929,0.000000,-0.931531,-0.439056,-0.807489,-0.180630,0.000000,1.537022
5,123706,141208,0,0,0,3.0,6.0,5.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,172304,141227,0,1,0,3.0,6.0,4.0,0.0,0.000000,...,1.039622,-0.923449,0.201810,0.000000,-0.767675,-1.481805,0.236039,-0.180630,0.000000,-1.455272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,1325400,3353235,0,0,0,4.0,6.0,5.0,0.0,-0.034765,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
171173,1528627,3353237,0,0,0,4.0,6.0,4.0,0.0,2.104829,...,0.000000,0.928756,-0.279924,0.919783,0.000000,0.000000,0.236039,-0.239808,-0.213987,0.000000
171174,1546299,3353251,1,1,0,1.0,1.0,1.0,0.0,-0.784374,...,-1.275313,0.000000,0.509423,0.000000,-0.592114,0.082318,0.145298,1.801835,0.000000,1.537022
171175,2085296,3353254,0,0,0,4.0,6.0,5.0,0.0,0.191658,...,0.000000,-0.084168,0.000000,0.000000,0.000000,0.000000,0.000000,-0.259534,0.000000,0.000000


In [98]:
apacheapsvar.to_csv(os.path.join(output_path, 'apacheapsvar_tmp.csv'), index=False)